In [ ]:
'''
In this notebook, we will use a trained neural network to estimate the pooled occupancy maps from the pooled llr sensor maps.
'''
import torch
import numpy as np
import scipy
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/gdrive')
from dense_ed import DenseED
%matplotlib inline

In [ ]:
def make_larger_sensors(sensor_map):
    '''
    This is a function we'll use to help us visualize the sensor locations
    '''
    max_x = sensor_map.size(dim=1)
    max_y = sensor_map.size(dim=0)
    out_sensor_map = torch.zeros(max_y, max_x)
    for y in range(sensor_map.size(dim=0)):
        for x in range(sensor_map.size(dim=1)):
            if sensor_map[y, x]:
                out_sensor_map[y, x] = 1

                back_y = y - 1 >= 0
                for_y = y + 1 < max_y

                back_x = x - 1 >= 0
                for_x = x + 1 < max_x

                if back_y:
                    out_sensor_map[y - 1, x] = 1

                if back_x:
                    out_sensor_map[y, x - 1] = 1

                if for_y:
                    out_sensor_map[y + 1, x] = 1

                if for_x:
                    out_sensor_map[y, x + 1] = 1
    return out_sensor_map

In [ ]:
'''
LOAD BACK IN OUR DATA FROM THE PREVIOUS NOTEBOOKS
'''
import os
power_maps = torch.load(f'{os.getcwd()}/power_maps.pt')
pooled_occupancy_maps = torch.load(f'{os.getcwd()}/pooled_occupancy_maps.pt')
pooled_llr_sensor_maps = torch.load(f'{os.getcwd()}/pooled_llr_sensor_maps.pt')

# Peek the maps
i_map = 2
pooled_viz_llr_sensors_H1 = make_larger_sensors((pooled_llr_sensor_maps[i_map] > 0).to(torch.float32))
pooled_viz_llr_sensors_H0 = make_larger_sensors((pooled_llr_sensor_maps[i_map] < 0).to(torch.float32))

fig, axs = plt.subplots(1, 3)
axs[0].imshow(torch.flip(power_maps[i_map], dims=[0]))
axs[0].axis('off')
axs[0].set_title('Power [dBm]')

axs[1].imshow(torch.flip(pooled_occupancy_maps[i_map], dims=[0]), vmin=0, vmax=1, interpolation='nearest')
axs[1].axis('off')
axs[1].set_title('Occupancy')

axs[2].imshow(torch.flip(pooled_viz_llr_sensors_H1 - pooled_viz_llr_sensors_H0, dims=[0]), vmin=-1, vmax=1, cmap='gray')
axs[2].axis("off")
axs[2].set_title('LLR Sensors')
plt.show()

In [ ]:
'''
LOAD THE TRAINED NEURAL NETWORK
This neural network is Dense Encoder Decoder Convolutional Neural Network
'''
model_loc = f'{os.getcwd()}/models/chicago/model_epoch500.pth'
model = DenseED(in_batch_norm=True, out_conv_ker_size=5)
model.load_state_dict(torch.load(model_loc, map_location=torch.device('cpu')))
model.eval()

In [ ]:
'''
INFERENCE THE NEURAL NETWORK 
Now we feed the neural network pooled llr sensor maps we generated in the previous Jupyter Notebook. This will create the pooled occupancy estimates.
'''
pooled_occupancy_map_estimates = torch.sigmoid(model(pooled_llr_sensor_maps[:, None])).detach() > 0.5
pooled_occupancy_map_estimates = pooled_occupancy_map_estimates[:, 0, :, :]

In [ ]:
'''
VIEW THE NEURAL NETWORK POOLED OCCUPANCY MAP ESTIMATES
'''
i_map = 0
pooled_viz_llr_sensors_H1 = make_larger_sensors((pooled_llr_sensor_maps[i_map] > 0).to(torch.float32))
pooled_viz_llr_sensors_H0 = make_larger_sensors((pooled_llr_sensor_maps[i_map] < 0).to(torch.float32))

accuracy = torch.mean(pooled_occupancy_map_estimates[i_map] == pooled_occupancy_maps[i_map], dtype=torch.float32).item()

fig, axs = plt.subplots(2, 2)
axs[0][0].imshow(torch.flip(power_maps[i_map], dims=[0]))
axs[0][0].axis('off')
axs[0][0].set_title('Power Map [dBm]')

axs[0][1].imshow(torch.flip(pooled_viz_llr_sensors_H1 - pooled_viz_llr_sensors_H0, dims=[0]), vmin=-1, vmax=1, cmap='gray')
axs[0][1].axis("off")
axs[0][1].set_title('LLR Sensors Map')

axs[1][0].imshow(torch.flip(pooled_occupancy_maps[i_map], dims=[0]), vmin=0, vmax=1, interpolation='nearest')
axs[1][0].axis('off')
axs[1][0].set_title('Occupancy Map')

axs[1][1].imshow(torch.flip(pooled_occupancy_map_estimates[i_map], dims=[0]), vmin=0, vmax=1, interpolation='nearest')
axs[1][1].axis('off')
axs[1][1].set_title(f'Estimate, Acc: {round(accuracy * 100, 1)}%')
plt.show()
